In [14]:
import pandas as pd
import csv
from itertools import combinations

data=pd.read_csv('/content/data1.csv')
minsupport=float(input("Enter support value :- "))
minsupport=minsupport*len(data)
minconfidence=float(input("Enter confidence value :- "))

items=[]#list to store all items of data as a lists of list
for i in range(0,len(data)):
     items.append([str(data.values[i,j]) for j in range(0,len(data.values[0]))])

count=[dict() for x in range(len(data.values[0])+1)]

s=[]#will store value of support for each element
for i in items:
     for j in i:
          s.append(j)

for i in s:
     #if item is present, inc by 1
     if i in count[1]:
          count[1][i]=count[1][i]+1
     else:
          count[1][i]=1

#remove items less than min support
for i in count[1].copy():
     if(count[1][i]<minsupport):
          count[1].pop(i)
count[1].pop('nan')

a=list(count[1]) #storing items > min support
item=[list() for i in range(len(data))]
c=0
for i in range(0,len(items)):
     for j in range(len(items[i])):
          if(a.__contains__(items[i][j])!=0):
             item[i].append(items[i][j])

#defining sort()fn to list item according to support
def sort(a):
     for i in range(len(a)-1):
          for j in range(len(a)-i-1):
               if(count[1][a[j]]<count[1][a[j+1]]):
                    a[j],a[j+1]=a[j+1],a[j]

#sorting in descending order
for i in range(0,len(data)):
     if(len(item[i])>1):
          sort(item[i])

#for FP tree linked generation 
class tree:
     def __init__(self,name,support,parent):
          self.name=name
          self.support=support
          self.nodeLink=None
          self.parent=parent
          self.child=[]

def ispresentchild(node,name):
     p=-1
     for i in node.child:
          p=p+1
          if(i.name)==name:
            return p
     return -1

#reference to last/first occurence of item for linked list
lastoccurence=count[1].copy()
for i in lastoccurence:
     lastoccurence[i]=None

#create FP_Tree
root=tree("root",-1,None)
p=0
for i in item:
     current=root
     for j in range(len(i)):
          if(ispresentchild(current,i[j])>=0):
               current=current.child[ispresentchild(current,i[j])]
               current.support=current.support + 1
          else:
               child1=tree(i[j],1,current)
               current.child.append(child1)
               t=current
               current=current.child[ispresentchild(current,i[j])]
               current.parent=t
               if(lastoccurence[current.name]==None):
                    lastoccurence[current.name]=current
               else:
                    current.nodeLink=lastoccurence[current.name]
                    lastoccurence[current.name]=current

#item set from a row                    
def value(a):
     a=str(a)
     a=a[:-2]
     a=a[2:]
     a=a[:-1]
     return a

#get item set with suffix node
def pathfn(node,n):
     c=0
     support=node.support
     path=[]
     pathname=[]
     current=node

     #path from current to root
     while(current.parent!=None):
          path.append(current)
          pathname.append(current.name)
          current=current.parent
     path.remove(node)
     pathname.remove(node.name)
     candidatepath=[]
     temp_candidatepath=[]
     #generate combination of lenght n
     a=(list(combinations(pathname,n)))
     for j in a:
          temp_candidatepath.append(tuple(sorted(j)))
     #append the suffix 'node.name'
     for j in temp_candidatepath:
          j=list(j)
          j.append(node.name)
          candidatepath.append(sorted(j))
     #update counts of generated items
     for j in candidatepath:
          j=tuple(j)
          if j in count[n+1]:
               count[n+1][j]=count[n+1][j]+support
          else:
               count[n+1][j]=support

     if(node.nodeLink!=None):
          node=node.nodeLink
          pathfn(node,i)
          
                   
#check if itemset is frequent
def freq(n):
     f=0
     for i in count[n]:
          if(count[n][i]>=minsupport):
               f=1
     if(f==1):
          return 1
     else:
          return 0

#pathfn() for each item
for i in range(1,len(data.values[0])+1):
     if(freq(i)==1):
          for j in lastoccurence:
               pathfn(lastoccurence[j],i)

#remove items less than min support
for z in range(len(data.values[0])+1):
     for i in count[z].copy():
          if(count[z][i]<minsupport):
            count[z].pop(i)

#len of longest itemset
i=2
while(len(count[i-1])!=0):
     i=i+1
q=i-2

#find maximal and closed item set
maximal=[]
closed=[]
for i in range(1,q):
     for j in count[i]:
          fmax=0
          fclosed=0
          for k in count[i+1]:
               a=set(list([j]))
               b=set(list(k))
               #set is maximal if no immediate super set is frequent
               if(a.intersection(b)==a):
                    fmax=1
                    #set is closed if none immediate superset have equal support
                    if(count[i][j]==count[i+1][k]):
                         fclosed=1
          if(fmax==0):
               maximal.append(i)
          if(fclosed==0):
               closed.append(j)
#All sets at the top of the tree are automatically maximal and closed
for i in count[q]:
     maximal.append(i)
     closed.append(i)

#Generate Association Rule
print("Association Rules are :- ")
abc=count.copy()
for i in range (q,0,-1):
     for j in abc[i]:
          for k in range(i-1,0,-1):
               itemset=list(combinations(list(j),k))
               #Traverse through set of all combination of antecendants
               for n in itemset:
                    #sort to prevent duplicacy
                    r=tuple(sorted(set(j).difference(set(n))))
                    l= len(n)
                    if(l==1):
                         n=value(n)
                         l=1
                    if(len(r)==1):
                         r2=value(r)
                    if(n!=None):
                         #if conf>minconf then print
                         if((abc[len(j)][j]/abc[l][n])>=minconfidence):
                              if(closed.__contains__((n))):
                                   c=c+1
                                   if(len(r)==1):
                                      print(n,"(",abc[l][n],")","-->",r2,"(",abc[len(r)][r2],")","confidence(",(abc[len(j)][j]/abc[l][n]),")")
                                   else:
                                        print(n,"(",abc[l][n],")","-->",r,"(",abc[len(r)][r],")","confidence(",(abc[len(j)][j]/abc[l][n]),")")
                                        

Enter support value :- 0.5
Enter confidence value :- 1
Association Rules are :- 
('a', 'c', 'f') ( 3 ) --> m ( 3 ) confidence( 1.0 )
('a', 'c', 'm') ( 3 ) --> f ( 4 ) confidence( 1.0 )
('a', 'f', 'm') ( 3 ) --> c ( 4 ) confidence( 1.0 )
('c', 'f', 'm') ( 3 ) --> a ( 3 ) confidence( 1.0 )
('a', 'c') ( 3 ) --> ('f', 'm') ( 3 ) confidence( 1.0 )
('a', 'f') ( 3 ) --> ('c', 'm') ( 3 ) confidence( 1.0 )
('a', 'm') ( 3 ) --> ('c', 'f') ( 3 ) confidence( 1.0 )
('c', 'f') ( 3 ) --> ('a', 'm') ( 3 ) confidence( 1.0 )
('c', 'm') ( 3 ) --> ('a', 'f') ( 3 ) confidence( 1.0 )
('f', 'm') ( 3 ) --> ('a', 'c') ( 3 ) confidence( 1.0 )
('a', 'c') ( 3 ) --> f ( 4 ) confidence( 1.0 )
('a', 'f') ( 3 ) --> c ( 4 ) confidence( 1.0 )
('c', 'f') ( 3 ) --> a ( 3 ) confidence( 1.0 )
('a', 'c') ( 3 ) --> m ( 3 ) confidence( 1.0 )
('a', 'm') ( 3 ) --> c ( 4 ) confidence( 1.0 )
('c', 'm') ( 3 ) --> a ( 3 ) confidence( 1.0 )
('a', 'f') ( 3 ) --> m ( 3 ) confidence( 1.0 )
('a', 'm') ( 3 ) --> f ( 4 ) confidence( 1.0 )